In [21]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from typing import List
from IPython.display import Markdown, display
from openai import OpenAI
import json

In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [5]:
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [10]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [55]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [36]:
# sample_answer = """ {
  "brochure": {
    "title": "Welcome to Shepperton Open Water Swimming",
    "introduction": "Experience the tranquility and thrill of open water swimming at Shepperton Open Water Swim, where every swim is special. Our dedicated lake provides a natural setting perfect for adult swimmers of all levels.",
    "venue": {
      "headline": "Not Just Any Lake",
      "description": "We've moved to our winter venue, Shepperton Splash, located at New Road, Shepperton, TW17 0QQ. Enjoy amenities like wood-fired saunas, hot tubs, changing rooms, covered seating, a coffee van, and an on-site kit shop."
    },
    "safety_info": {
      "headline": "Safety is Our Priority",
      "description": "At Shepperton Open Water Swim, we prioritize safety with controlled swimmer numbers and strict adherence to safety procedures. Please respect our safety team's instructions."
    },
    "swimming_sessions": {
      "headline": "Open for Summer 2024",
      "details": "Our family-run facility offers specific swim session times. New swimmers must book an induction, which is mandatory to ensure safety and confidence in the open water."
    },
    "induction_swim": {
      "description": "New Swimmers are required to participate in an induction swim to familiarize themselves with our safety rules and complete a swim test. After successfully completing the induction, you will receive login details for future bookings.",
      "booking_info": "To book your induction swim, contact us through email or social media."
    },
    "operational_hours": {
      "headline": "Opening Times",
      "schedule": {
        "Saturday": "7-11 AM",
        "Sunday": "7-11 AM",
        "Weekdays": "CLOSED"
      }
    },
    "location": {
      "headline": "Find Us",
      "address": "Ferris Meadow Lake, Ferry Lane, Shepperton, Middlesex, TW17 9LH",
      "notes": "Please note that the lake is private property and accessible only during opening times."
    },
    "contact_info": {
      "headline": "Get in Touch",
      "email": "info@sheppertonopenwaterswim.co.uk",
      "phone": "07531 745133",
      "social_media": "Follow us on our social media channels for updates."
    },
    "coaching": {
      "headline": "Interested in Coaching?",
      "description": "We offer individual or group coaching for all skill levels. Contact us to arrange your coaching sessions."
    },
    "footer": "All content copyright © sheppertonopenwaterswim.co.uk | All rights reserved"
  }
}
"""

In [41]:
system_prompt = f"You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in JSON. \
following this example:  {sample_answer}"

In [48]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 20,000 characters
    return user_prompt

In [13]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [27]:
ws_string = "https://sheppertonopenwaterswim.co.uk/"

In [56]:
create_brochure("Shepperton Open Water Swimming", ws_string)

Found links: {'links': [{'type': 'home page', 'url': 'https://sheppertonopenwaterswim.co.uk/'}, {'type': 'contact page', 'url': 'https://sheppertonopenwaterswim.co.uk/#contact'}, {'type': 'location page', 'url': 'https://sheppertonopenwaterswim.co.uk/#location'}, {'type': 'opening times page', 'url': 'https://sheppertonopenwaterswim.co.uk/#opening-times'}, {'type': 'swimming page', 'url': 'https://sheppertonopenwaterswim.co.uk/#swimming'}, {'type': 'Facebook page', 'url': 'https://en-gb.facebook.com/SheppertonOpenWaterSwim/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/sheppertonswim'}, {'type': 'Instagram page', 'url': 'https://www.instagram.com/sheppertonopenwaterswim/'}]}


```markdown
# Welcome to Shepperton Open Water Swimming

## Overview
Shepperton Open Water Swimming (SOWS) offers a unique experience for adult swimmers looking to enjoy open water in a natural environment. As a family-run venue, we prioritize safety and provide a comfortable atmosphere for all levels of swimmers. Our current winter venue at Shepperton Splash features wood-fired saunas, hot tubs, changing rooms, and an on-site kit shop—perfect for both relaxation and preparation before your swim.

## Company Culture
At Shepperton Open Water Swimming, we foster a welcoming and inclusive culture. Our team is passionate about open water swimming and is dedicated to ensuring you have a safe and enjoyable experience. Our commitment to customer safety is reflected in our thorough induction procedure for all new swimmers. We believe in building a community around the love of swimming and encourage camaraderie among participants.

## Safety First
Your safety is our top priority. 
- **Induction Required**: All new swimmers must undergo a brief induction for safety training and swim testing, ensuring they're comfortable in an open-water setting.
- **Controlled Environment**: We operate on a booking-only basis, with controlled numbers for each swim session to maintain adequate spacing among swimmers. 

No booking, no swim—so please respect our guidelines.

## Customer Experience
We cater to adult swimmers of all experience levels and ensure proper guidance and support throughout your swimming journey.
- **Flexible Booking**: Registered swimmers can book sessions easily via our Wellness Living Achieve app.
- **Only for Adults**: Our venue is designed strictly for adults (18+), while under-18s must swim with a coach.

## Location
Located at Ferris Meadow Lake, Shepperton, Middlesex, our private facility ensures a tranquil swimming experience. 
- **Address**: Ferris Meadow Lake, Ferry Lane, Shepperton, Middlesex, TW17 9LH
- **Winter Venue**: Shepperton Splash, New Road, Shepperton, TW17 0QQ (Includes facilities like saunas and hot tubs)

## Careers and Jobs
Shepperton Open Water Swimming is always on the lookout for enthusiastic individuals looking to immerse themselves in the world of open water swimming. If you are passionate about aquatics and want to join a supportive team dedicated to providing excellent customer service, we encourage you to get in touch and explore potential career opportunities with us.

## Contact Us
Ready to dive in? Book your induction today or reach out for more information through our contact methods:
- **Email**: [info@sheppertonopenwaterswimming.com](mailto:info@sheppertonopenwaterswimming.com)
- **Phone**: 07531 745133
- **Social Media**: Available on our website

## Join Us
Experience the beauty of open water swimming with Shepperton Open Water Swimming for Summer 2024 and beyond. We look forward to welcoming you to our community!

---
```
This brochure provides a comprehensive overview of Shepperton Open Water Swimming, highlighting the company's culture, commitment to safety, customer experience, location, and career opportunities.